# Experiment 4: Fixed 10-Fold Cross-Validation on FPB

**Aim:** Properly replicate the FinBERT paper's evaluation protocol — 10 stratified 90/10 splits on FPB `sentences_50agree`, with model **re-initialized from scratch each fold**.

**Fixes over earlier k-fold attempt:**
- Re-initialize model + LoRA from pretrained weights every fold (was accumulating weights)
- Use `StratifiedKFold` instead of random shuffle (ensures label balance)
- Use `transformers + peft` (consistent with NB01/NB02)
- Use NB01/NB02 hyperparams: LoRA r=16/alpha=32, lr=2e-4, effective batch 32, cosine schedule
- `load_best_model_at_end=True` by eval_loss
- Per-fold output directories to avoid checkpoint overwrites
- Collect per-fold accuracy + macro F1 + per-class metrics, report mean ± std

**Runtime:** ~3-4 hours on T4 (5 epochs × 10 folds)

## 1. Setup & Installation

In [ ]:
# setup

In [ ]:
%%capture
!pip install -q "datasets>=3.4.1,<4.0.0" scikit-learn matplotlib seaborn peft accelerate transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn.functional as F
from transformers import (
    TrainingArguments, Trainer, AutoModelForSequenceClassification,
    AutoTokenizer, training_args,
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc

NUM_CLASSES = 3
LABEL_NAMES = ["NEGATIVE", "NEUTRAL", "POSITIVE"]
MODEL_NAME = "answerdotai/ModernBERT-base"
N_FOLDS = 10
EPOCHS_PER_FOLD = 5
SEED = 42

## 2. Load FPB sentences_50agree

Use the full 4,846-sample dataset for 10-fold cross-validation. Each fold uses 90% for training, 10% for testing.

In [ ]:
fpb = load_dataset("financial_phrasebank", "sentences_50agree", trust_remote_code=True)["train"]

texts = fpb["sentence"]
labels = np.array(fpb["label"])

print(f"FPB sentences_50agree: {len(texts):,} samples")
print(f"Label distribution:")
for i, name in enumerate(LABEL_NAMES):
    count = (labels == i).sum()
    print(f"  {name}: {count} ({count/len(labels):.1%})")

## 3. Define Training Function

Creates a **fresh** model + LoRA adapter from pretrained weights each call. This is the critical fix — no weight accumulation across folds.

In [ ]:
def train_fold(train_texts, train_labels, test_texts, test_labels, fold_idx, epochs=EPOCHS_PER_FOLD):
    """Train a fresh ModernBERT+LoRA model on one fold and evaluate.

    Args:
        train_texts: List of training texts.
        train_labels: Array of integer labels for training.
        test_texts: List of test texts.
        test_labels: Array of integer labels for testing.
        fold_idx: Fold number (for output directory naming).
        epochs: Number of training epochs.

    Returns:
        dict with accuracy, macro_f1, per_class_f1, per_class_precision, per_class_recall.
    """
    print(f"\n{'='*60}")
    print(f"FOLD {fold_idx + 1}/{N_FOLDS}")
    print(f"Train: {len(train_texts)}  |  Test: {len(test_texts)}")
    print(f"{'='*60}")

    # Fresh model every fold
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_CLASSES,
        torch_dtype=torch.float32,
        attn_implementation="sdpa",
    )
    model.gradient_checkpointing_enable()
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["Wqkv", "out_proj", "Wi", "Wo"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_CLS,
    )
    model = get_peft_model(model, lora_config)
    model = model.cuda()

    if fold_idx == 0:
        model.print_trainable_parameters()

    # Create datasets with one-hot labels (consistent with NB01/NB02)
    train_ds = Dataset.from_dict({
        "text": train_texts,
        "labels": [np.eye(NUM_CLASSES)[l].tolist() for l in train_labels],
    })
    test_ds = Dataset.from_dict({
        "text": test_texts,
        "labels": [np.eye(NUM_CLASSES)[l].tolist() for l in test_labels],
    })

    def tokenize_function(examples):
        return tokenizer(examples["text"])

    train_tok = train_ds.map(tokenize_function, batched=True)
    test_tok = test_ds.map(tokenize_function, batched=True)

    output_dir = f"trainer_output_fold_{fold_idx}"

    trainer = Trainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=train_tok,
        eval_dataset=test_tok,
        args=TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=8,
            gradient_accumulation_steps=4,
            warmup_steps=10,
            fp16=True,
            bf16=False,
            optim=training_args.OptimizerNames.ADAMW_TORCH,
            learning_rate=2e-4,
            weight_decay=0.001,
            lr_scheduler_type="cosine",
            seed=3407,
            num_train_epochs=epochs,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            save_strategy="epoch",
            eval_strategy="epoch",
            logging_strategy="epoch",
            gradient_checkpointing=True,
            report_to="none",
        ),
        compute_metrics=lambda eval_pred: {
            "accuracy": accuracy_score(
                eval_pred[1].argmax(axis=-1), eval_pred[0].argmax(axis=-1)
            )
        },
    )

    trainer.train()
    model = model.cuda().eval()

    # Run inference on test fold
    all_preds = []
    with torch.no_grad():
        for i in range(0, len(test_texts), 32):
            batch = test_texts[i : i + 32]
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = {k: v.cuda() for k, v in inputs.items()}
            logits = model(**inputs).logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            all_preds.extend(preds)

    y_pred = np.array(all_preds)
    y_true = test_labels

    acc = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average="macro")
    per_class_f1 = f1_score(y_true, y_pred, average=None)
    per_class_prec = f1_score(y_true, y_pred, average=None)  # placeholder
    per_class_rec = f1_score(y_true, y_pred, average=None)  # placeholder

    # Get proper per-class metrics from classification_report
    from sklearn.metrics import precision_score, recall_score
    per_class_prec = precision_score(y_true, y_pred, average=None)
    per_class_rec = recall_score(y_true, y_pred, average=None)

    report = classification_report(y_true, y_pred, target_names=LABEL_NAMES)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\nFold {fold_idx + 1} — Accuracy: {acc:.4f}  Macro F1: {macro_f1:.4f}")
    print(report)

    # Cleanup to free GPU memory
    del model, trainer
    gc.collect()
    torch.cuda.empty_cache()

    return {
        "fold": fold_idx + 1,
        "accuracy": acc,
        "macro_f1": macro_f1,
        "per_class_f1": per_class_f1,
        "per_class_precision": per_class_prec,
        "per_class_recall": per_class_rec,
        "confusion_matrix": cm,
        "n_train": len(train_texts),
        "n_test": len(test_texts),
    }

## 4. Run 10-Fold Cross-Validation

Use `StratifiedKFold` to ensure balanced label distribution in each fold.

In [ ]:
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

fold_results = []

for fold_idx, (train_idx, test_idx) in enumerate(skf.split(texts, labels)):
    train_texts_fold = [texts[i] for i in train_idx]
    train_labels_fold = labels[train_idx]
    test_texts_fold = [texts[i] for i in test_idx]
    test_labels_fold = labels[test_idx]

    result = train_fold(
        train_texts_fold, train_labels_fold,
        test_texts_fold, test_labels_fold,
        fold_idx=fold_idx,
    )
    fold_results.append(result)

print(f"\nAll {N_FOLDS} folds complete!")

## 5. Results Summary

In [ ]:
# Per-fold results table
results_df = pd.DataFrame([
    {
        "Fold": r["fold"],
        "Accuracy": r["accuracy"],
        "Macro F1": r["macro_f1"],
        "F1 Negative": r["per_class_f1"][0],
        "F1 Neutral": r["per_class_f1"][1],
        "F1 Positive": r["per_class_f1"][2],
        "Train Size": r["n_train"],
        "Test Size": r["n_test"],
    }
    for r in fold_results
])

print("=" * 90)
print("10-FOLD CROSS-VALIDATION RESULTS — ModernBERT-base + LoRA on FPB sentences_50agree")
print("=" * 90)
print(results_df.to_string(index=False, float_format="%.4f"))

# Summary statistics
print(f"\n{'='*90}")
print("SUMMARY (mean ± std)")
print(f"{'='*90}")
for metric in ["Accuracy", "Macro F1", "F1 Negative", "F1 Neutral", "F1 Positive"]:
    mean = results_df[metric].mean()
    std = results_df[metric].std()
    print(f"  {metric:15s}: {mean:.4f} ± {std:.4f}")

print(f"\n  Min Accuracy:  {results_df['Accuracy'].min():.4f} (Fold {results_df.loc[results_df['Accuracy'].idxmin(), 'Fold']})")
print(f"  Max Accuracy:  {results_df['Accuracy'].max():.4f} (Fold {results_df.loc[results_df['Accuracy'].idxmax(), 'Fold']})")

## 6. Comparison with Published Baselines

Compare our in-domain cross-validation results against published baselines that also use in-domain FPB evaluation.

In [ ]:
mean_acc = results_df["Accuracy"].mean()
std_acc = results_df["Accuracy"].std()
mean_f1 = results_df["Macro F1"].mean()
std_f1 = results_df["Macro F1"].std()

comparison = pd.DataFrame([
    {"Model": "LSTM+ELMo", "Accuracy": "0.7500", "Macro F1": "0.7000",
     "Protocol": "In-domain (80/20)", "Source": "Araci 2019"},
    {"Model": "ULMFit", "Accuracy": "0.8300", "Macro F1": "0.7900",
     "Protocol": "In-domain (80/20)", "Source": "Araci 2019"},
    {"Model": "ProsusAI/finbert", "Accuracy": "0.8600", "Macro F1": "0.8400",
     "Protocol": "In-domain (80/20)", "Source": "Araci 2019"},
    {"Model": "FinBERT-FinVocab", "Accuracy": "0.8720", "Macro F1": "—",
     "Protocol": "In-domain (90/10, 10-run avg)", "Source": "Yang et al. 2020"},
    {"Model": "finbert-lc", "Accuracy": "0.8900", "Macro F1": "0.8800",
     "Protocol": "In-domain", "Source": "2024"},
    {"Model": "ModernBERT+LoRA (ours)",
     "Accuracy": f"{mean_acc:.4f} ± {std_acc:.4f}",
     "Macro F1": f"{mean_f1:.4f} ± {std_f1:.4f}",
     "Protocol": f"In-domain 10-fold CV", "Source": "This experiment"},
])

print("=" * 95)
print("COMPARISON — ModernBERT vs Published Baselines on FPB sentences_50agree")
print("=" * 95)
print(comparison.to_string(index=False))
print("\nNote: All baselines use in-domain FPB evaluation. Our result is 10-fold stratified CV.")

## 7. Visualizations

In [ ]:
# Per-fold accuracy bar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: accuracy per fold
ax = axes[0]
bars = ax.bar(results_df["Fold"], results_df["Accuracy"], color="#2196F3", edgecolor="white")
ax.axhline(y=mean_acc, color="red", linestyle="--", label=f"Mean: {mean_acc:.4f}")
ax.fill_between(
    [0.5, N_FOLDS + 0.5], mean_acc - std_acc, mean_acc + std_acc,
    alpha=0.15, color="red", label=f"±1 std: {std_acc:.4f}"
)
ax.set_xlabel("Fold")
ax.set_ylabel("Accuracy")
ax.set_title("Accuracy per Fold")
ax.set_xticks(range(1, N_FOLDS + 1))
ax.legend()
ax.set_ylim(0.85, 1.0)

# Right: macro F1 per fold
ax = axes[1]
bars = ax.bar(results_df["Fold"], results_df["Macro F1"], color="#4CAF50", edgecolor="white")
ax.axhline(y=mean_f1, color="red", linestyle="--", label=f"Mean: {mean_f1:.4f}")
ax.fill_between(
    [0.5, N_FOLDS + 0.5], mean_f1 - std_f1, mean_f1 + std_f1,
    alpha=0.15, color="red", label=f"±1 std: {std_f1:.4f}"
)
ax.set_xlabel("Fold")
ax.set_ylabel("Macro F1")
ax.set_title("Macro F1 per Fold")
ax.set_xticks(range(1, N_FOLDS + 1))
ax.legend()
ax.set_ylim(0.85, 1.0)

plt.suptitle("10-Fold CV — ModernBERT+LoRA on FPB sentences_50agree", fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig("kfold_cv_results.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Per-class F1 box plot across folds
fig, ax = plt.subplots(figsize=(8, 5))

class_f1_data = {
    name: [r["per_class_f1"][i] for r in fold_results]
    for i, name in enumerate(LABEL_NAMES)
}

bp = ax.boxplot(
    class_f1_data.values(),
    labels=class_f1_data.keys(),
    patch_artist=True,
    boxprops=dict(facecolor="#90CAF9"),
)

ax.set_ylabel("F1 Score")
ax.set_title("Per-Class F1 Distribution Across 10 Folds")
ax.set_ylim(0.7, 1.0)
plt.tight_layout()
plt.savefig("kfold_class_f1_boxplot.png", dpi=150, bbox_inches="tight")
plt.show()

## 8. Sanity Check: Training Loss at Epoch 1 Across Folds

Verify that model re-initialization is working correctly. If weights were accumulating, epoch-1 loss would decrease monotonically across folds. With proper re-initialization, epoch-1 loss should be similar across all folds.

In [ ]:
print("Verification: Training loss at epoch 1 should be similar across all folds.")
print("If it decreases monotonically, model weights are leaking between folds.")
print("\nCheck the training logs above — epoch 1 loss for each fold should be ~1.0-1.1.")

print(f"\n{'='*60}")
print("FINAL RESULT")
print(f"{'='*60}")
print(f"ModernBERT-base + LoRA — 10-fold CV on FPB sentences_50agree")
print(f"Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Macro F1: {mean_f1:.4f} ± {std_f1:.4f}")